# Modelo de recomendação

## Setup

In [1]:
import pandas as pd
import numpy as np
import joblib

from ipywidgets import interact
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Pré-processamento dos dados

In [2]:
X = pd.read_parquet('../data/estaticos_market.parquet')

In [3]:
X.sample(5)

,id,fl_matriz,de_natureza_juridica,sg_uf,natureza_juridica_macro,de_ramo,setor,idade_empresa_anos,idade_emp_cat,fl_me,fl_sa,fl_epp,fl_mei,fl_ltda,dt_situacao,fl_st_especial,fl_email,fl_telefone,fl_rm,nm_divisao,nm_segmento,fl_spa,fl_antt,fl_veiculo,fl_optante_simples,vl_total_veiculos_pesados_grupo,vl_total_veiculos_leves_grupo,fl_optante_simei,sg_uf_matriz,de_saude_tributaria,de_saude_rescencia,nu_meses_rescencia,de_nivel_atividade,fl_simples_irregular,empsetorcensitariofaixarendapopulacao,nm_meso_regiao,nm_micro_regiao,fl_passivel_iss,qt_socios,qt_socios_pf,qt_socios_pj,idade_media_socios,idade_maxima_socios,idade_minima_socios,qt_socios_st_regular,qt_socios_masculino,qt_socios_feminino,qt_coligados,qt_coligados_matriz,qt_coligados_ativo,qt_coligados_baixada,qt_coligados_inapta,qt_coligados_suspensa,qt_coligados_nula,qt_coligados_sa,qt_coligados_me,qt_coligados_mei,qt_coligados_ltda,qt_coligados_epp,qt_coligados_norte,qt_coligados_sul,qt_coligados_nordeste,qt_coligados_centro,qt_coligados_sudeste,qt_coligados_exterior,qt_coligados_industria,qt_coligados_agropecuaria,qt_coligados_comercio,qt_coligados_serviço,qt_coligados_ccivil,qt_coligados_atividade_alto,qt_coligados_atividade_medio,qt_coligados_atividade_baixo,qt_coligados_atividade_mt_baixo,qt_coligados_atividade_inativo,de_faixa_faturamento_estimado,de_faixa_faturamento_estimado_grupo,vl_faturamento_estimado_aux,vl_faturamento_estimado_grupo_aux,qt_filiais
163704,6ba5b2974388ce54ddbc38f6ee3023a67e5c385f311c07...,1,SOCIEDADE SIMPLES PURA,RO,ENTIDADES EMPRESARIAIS,SERVICOS DE SAUDE,SERVIÇO,0.189041,<= 1,0,0,0,0,0,2018-08-20,0,1,1,NAO,ATIVIDADES DE ATENCAO A SAUDE HUMANA,SAUDE HUMANA E SERVICOS SOCIAIS,False,False,False,None,0.0,0.0,None,RO,None,None,NaN,None,False,1681.00,None,None,True,5.0,5.0,0.0,35.0,39.0,32.0,3.0,2.0,1.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,None,None,NaN,NaN,0
42384,3c06d5a5eb3aa7c80d8b6cd78bd31be30a0a289d4cf0d9...,1,EMPRESARIO INDIVIDUAL,AM,OUTROS,SERVICOS DE ALOJAMENTO/ALIMENTACAO,SERVIÇO,2.068493,1 a 5,0,0,0,1,0,2016-10-03,0,1,1,SIM,ALIMENTACAO,ALOJAMENTO E ALIMENTACAO,False,False,False,True,0.0,0.0,True,AM,VERDE,ACIMA DE 1 ANO,22.0,ALTA,False,3071.98,CENTRO AMAZONENSE,MANAUS,True,1.0,1.0,0.0,30.0,30.0,30.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"ATE R$ 81.000,00","ATE R$ 81.000,00",50000.0,50000.0,0
85960,1d2a785b9a91712d7b1ed131efb179c16b0b35f29daf55...,1,SOCIEDADE EMPRESARIA LIMITADA,RN,ENTIDADES EMPRESARIAIS,INDUSTRIA DA CONSTRUCAO,CONSTRUÇÃO CIVIL,2.268493,1 a 5,0,0,0,0,0,2016-07-22,0,1,0,SIM,CONSTRUCAO DE EDIFICIOS,CONSTRUCAO,False,False,False,False,0.0,0.0,False,RN,VERDE,ACIMA DE 1 ANO,24.0,ALTA,False,NaN,LESTE POTIGUAR,NATAL,True,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"DE R$ 81.000,01 A R$ 360.000,00","DE R$ 81.000,01 A R$ 360.000,00",210000.0,210000.0,0
143044,712beadb857fe49a0028c1ef6dd9447df17310fa91b106...,0,EMPRESARIO INDIVIDUAL,PI,OUTROS,COMERCIO VAREJISTA,COMERCIO,4.293151,1 a 5,0,0,0,0,0,2014-07-14,0,0,1,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,False,False,False,True,1.0,0.0,False,PI,VERDE,ACIMA DE 1 ANO,21.0,MEDIA,False,787.99,NORTE PIAUIENSE,LITORAL PIAUIENSE,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"DE R$ 360.000,01 A R$ 1.500.000,00","DE R$ 360.000,01 A R$ 1.500.000,00",370915.2,1112745.6,2
298951,5153413f3a014bba2b8681ae611d4ed5f42d2d52ae6cb2...,1,EMPRESARIO INDIVIDUAL,PI,OUTROS,COMERCIO VAREJISTA,COMERCIO,49.816438,> 20,0,0,0,0,0,2005-11-03,0,0,0,NAO,COMERCIO VAREJISTA,COMERCIO; REPARACAO DE VEICULOS AUTOMOTORES E ...,False,False,False,False,0.0,0.0,False,PI,AMARELO,ACIMA DE 1 ANO,22.0,MEDIA,False,1960.54,NORTE PIAUIENSE,

## Features categóricas

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer

In [17]:
cat_features = [*X.select_dtypes(include='object').columns]
cat_features.remove('id')
cat_features.remove('dt_situacao')

cat_imputer = SimpleImputer(strategy='most_frequent', missing_values=None)
cat_encoder = OneHotEncoder(handle_unknown='ignore', sparse=True)

In [18]:
cat_transformer = Pipeline([
    ('cat_imp', cat_imputer),
    ('cat_enc', cat_encoder)
])

## Features numéricas

In [7]:
from sklearn.preprocessing import StandardScaler

In [8]:
# todas as features numéricas
num_features = [*X.select_dtypes(exclude='object').columns]

In [9]:
# features numéricas: 2 estratégias para dados faltantes, de acordo com a análise exploratória

# características da empresa
nu1_features = [i for i in num_features if str.startswith(i, 'fl_')]
nu1_imputer = SimpleImputer(strategy='most_frequent')
nu1_scaler = StandardScaler()

nu1_transformer = Pipeline([
    ('nu1_imp', nu1_imputer),
    ('nu1_sca', nu1_scaler)
])

# quantidades de coligadas
nu2_features = [i for i in num_features if str.startswith(i, 'qt_colig')]
nu2_imputer = SimpleImputer(strategy='constant', fill_value=0)

nu2_transformer = Pipeline([
    ('nu2_imp', nu2_imputer)
])

# demais numéricas
nu3_features = list(set(num_features) - set(nu1_features) - set(nu2_features))

nu3_imputer = SimpleImputer(strategy='mean')
nu3_scaler = StandardScaler()

nu3_transformer = Pipeline([
    ('nu3_imp', nu1_imputer),
    ('nu3_sca', nu1_scaler)
])


## Consolidado

In [10]:
from sklearn.compose import ColumnTransformer

In [19]:
data_transformer = ColumnTransformer([
    ('cat_tr', cat_transformer, cat_features),
    ('nu1_tr', nu1_transformer, nu1_features),
    ('nu2_tr', nu2_transformer, nu2_features),
    ('nu3_tr', nu3_transformer, nu3_features)
], remainder='drop')

# Modelo

In [12]:
from sklearn.neighbors import NearestNeighbors

In [13]:
knn_model = NearestNeighbors(metric='cosine', algorithm='auto', n_neighbors=10, n_jobs=-1)

In [20]:
pipe = Pipeline([
    ('data_tr', data_transformer),
    ('model', knn_model)
])

In [23]:
pipe.fit(X)

Pipeline(steps=[('data_tr',
                 ColumnTransformer(transformers=[('cat_tr',
                                                  Pipeline(steps=[('cat_imp',
                                                                   SimpleImputer(missing_values=None,
                                                                                 strategy='most_frequent')),
                                                                  ('cat_enc',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['de_natureza_juridica',
                                                   'sg_uf',
                                                   'natureza_juridica_macro',
                                                   'de_ramo', 'setor',
                                                   'idade_emp_cat', 'fl_rm',
                                                   'nm_divisao', 'nm_segme

# Pickling

In [24]:
# exporta o modelo treinado
joblib.dump(pipe, '../model.joblib')

['../model.joblib']